In [1]:
import pandas as pd
import numpy as np
import ast
import decimal
import psycopg2

In [2]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password=""
)

In [3]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [4]:
df_users = pd.read_csv("data/users.csv")

In [5]:
# Sort dataframe by date column
df_users = df_users.sort_values('date')

In [6]:
df_users.head()

,user_id,name,screen_name,date,twitter_join_date,location,description,verified,followers_count,friends_count,listed_count,favourites_count,language
73823,Peace🍁,False,16,11,0,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77882,Peace🍁,False,16,11,0,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79605,Peace🍁,False,16,11,0,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82930,Press agent.,False,354,1650,4,3400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89005,(Takipçi kaygısından çok insanlık kaygısı olan...,False,2944,1867,16,39708,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
try:
    for index, row in df_users.iterrows():
        cur.execute("""
        INSERT INTO twitter_users (user_id, name, screen_name, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (user_id, date) DO NOTHING
    """, (row['user_id'], row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language']))

    conn.commit()
    cur.close()
    conn.close()

except Exception as e:
    conn.rollback()
    cur.close()
    conn.close()
    print(e)


In [8]:
# Loop through rows of dataframe
# for i, row in df_users.iterrows():
#     # Check if user already exists in fact table
#     cur.execute("SELECT * FROM User_table_fact WHERE user = %s", (row['user'],))
#     user_fact = cur.fetchone()
    
#     # If user exists, update row in fact table
#     if user_fact:
#         cur.execute("UPDATE User_table_fact SET name = %s, screen_name = %s, date = %s, twitter_join_date = %s, location = %s, description = %s, verified = %s, followers_count = %s, friends_count = %s, listed_count = %s, favourites_count = %s, preferred_language = %s WHERE user_id = %s", (row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language'], user_fact[0]))
#         user_id = user_fact[0]
#     # If user does not exist, insert row into fact table
#     else:
#         cur.execute("INSERT INTO User_table_fact (user, name, screen_name, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING user_id", (row['user'], row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language']))
#         user_id = cur.fetchone()[0]
    
#     # Insert row into dim table
#     cur.execute("INSERT INTO User_table_dim (user_id, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s